## Shopee 2021 Hackathon - Address Elements Extraction

### The goal of this competition is to build a model to correctly extract Point of Interest (POI) Names and Street Names from unformatted Indonesia addresses.

### For example, 

In [1]:
import re
import pickle
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from nltk.util import everygrams,bigrams
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import NearestNeighbors
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold

In [2]:
########################################## CLASS DEFINITION FOR PREPROCESS LOGIC #######################

REPLACE_WITH_STRING = ","

class PreProcessLogic:
    def process(self, string: str):
        pass

class RukunTetanggaPreProcessLogic(PreProcessLogic):
    def process(self, string: str):
        return re.sub(r"\brt(\.)*(( )*[0-9]+)+", REPLACE_WITH_STRING, string)

class RukunWargaPreProcessLogic(PreProcessLogic):
    def process(self, string: str):
        return re.sub(r"\brw(\.)*(( )*[0-9]+)+", REPLACE_WITH_STRING, string)

class PostalCodePreProcessLogic(PreProcessLogic):
    def process(self, string: str):
        return re.sub(r"\b[0-9]{5}[^,]*", REPLACE_WITH_STRING, string)
    
class NumberCodePreprocessLogic(PreProcessLogic):
    def process(self, string: str):
        return re.sub(r"\bno[0-9\-]*\b(\.)*(( )*([a-z]{0,2}( )*[0-9\-]+[a-z]{0,2}|[0-9\-][a-z]{1})\b)*", REPLACE_WITH_STRING, string)


class PreprocessingHelper:
    def generate_all_n_grams(self, string: str):
        result = []
        for part in string.split(","):
            part = part.strip()
            result.extend([' '.join(x) for x in everygrams(part.split())])

        return np.asarray(result)  

    def generate_all_bi_grams(self, string: str):
        result = []
        if "/" in string:
            for part in string.split("/"):
                if part:
                    if " " not in part:
                        result.append(part)
                    else:
                        part = part.strip()
                        result.extend([' '.join(x) for x in bigrams(part.split())])
            return result
        else:
            str_list = string.split()
            if len(str_list) == 1:
                return str_list
            return [' '.join(x) for x in bigrams(str_list)]


    def generate_all_n_grams_labels(self, n_grams: list, labels: list): 
        labels_for_n_gram = np.zeros(shape=(len(n_grams),len(labels)))

        for i in range(len(n_grams)):
            for j in range(len(labels)):
                # use prefix matching for all words
                original_string = re.escape(n_grams[i]).replace("\ ", " ")
                prefix_regex = "^" + re.sub(r"\s+", r"[^\\s]*\\s+", original_string)
                prefix_match_result = re.search(prefix_regex, labels[j])
                if prefix_match_result:
                    labels_for_n_gram[i][j] = 1

        return labels_for_n_gram

    def preprocesing_step(self, ids: list, raw_address: list, answers: list):
        processing_logics = [RukunTetanggaPreProcessLogic(), RukunWargaPreProcessLogic(), PostalCodePreProcessLogic(), NumberCodePreprocessLogic()]

        processed_address = []

        for addr in raw_address:
            new_addr = addr
            for logic in processing_logics:
                new_addr = logic.process(new_addr)
            processed_address.append(new_addr)

        X = []
        Y = []
        for i in range(len(processed_address)):
            x = self.generate_all_n_grams(processed_address[i])
            X.extend([[ids[i], x_i] for x_i in x])
            y = []
            if i < len(answers):
                y = self.generate_all_n_grams_labels(x, answers[i].split("/"))
                Y.extend(y)

        return np.array(X), np.array(Y)

In [3]:
# define encoding functions

class bag_of_word_encoder:
    
    def __init__(self, kmean_n=None):
        self.kmean_n = kmean_n
        
        
    def get_unique_words(self, data_np):
        
        # get unique words
        unique_words = []
        for address in data_np:
            unique_words.extend(address.split(' '))
            
        unique_words = list(set(unique_words))
        
        # edge case: remove empty element
        #unique_words.remove('')
        
        return unique_words
            
    
    def get_feature_dimension(self, unique_words):
        
        # get unique character
        unique_character = set()
        for unique in unique_words:
            if len(unique) != 0:
                for u in list(unique):
                    unique_character.add(u)
        unique_character = list(unique_character)
                               
        # encode each word by the unique character
        X = self.one_hot_encoder(unique_words, unique_character = unique_character)
        
        return X, unique_character

    
    def one_hot_encoder(self, unique_words, unique_character=None):
        
        if unique_character == None:
            unique_character = self.unique_character_
        
        X = np.zeros((len(unique_words),len(unique_character)), dtype=int)
        for i, word in enumerate(unique_words):
            for u in list(word):
                c = [j for j, item in enumerate(unique_character) if item in set(u)]
                X[i,c[0]] = 1    
      
        return X
                               
    
    def train_Kmeans(self, X, n_init=10, max_iter=500):
        
        kmeans = KMeans(n_clusters=self.kmean_n, n_init=n_init, max_iter=max_iter)
        kmeans.fit(X)

        clusters_dimension = kmeans.cluster_centers_

        return clusters_dimension
    
    
    def build_dictionary(self, data_np, load_kmean=False):
        
        # get all unique words from data
        unique_words = self.get_unique_words(data_np)
         
        # get one-hot-encoder X
        X, unique_character = self.get_feature_dimension(unique_words)
    
        if load_kmean == False:                      
                               
            # perform k-mean clustering to clusters unique words
            print('Begin computing K-Means clustering...', end='')
            clusters_dimension = self.train_Kmeans(X)
            np.savetxt('kMeansCluster.out', clusters_dimension, delimiter=',')
            print('Complete!')
            
        else:
            # import kmean result
            clusters_dimension = np.loadtxt('kMeansCluster.out', delimiter=',')
        
        # save result in class
        self.clusters_dimension_ = clusters_dimension
        self.unique_words_ = unique_words
        self.unique_character_ = unique_character
    
    
    def compute_bow(self, data_np):
        
        bow_features = np.zeros((data_np.shape[0], self.clusters_dimension_.shape[0]))

        # fit the kmean model
        NN = NearestNeighbors(n_neighbors=1)
        NN.fit(self.clusters_dimension_)        
        
        for i, address in enumerate(data_np):
            words = address.split(' ')
            features = self.one_hot_encoder(words)
            
            # find the nearest neighbors
            _, indices = NN.kneighbors(features)
            
            # create bins
            bins = [i for i in range(0,self.clusters_dimension_.shape[0]+1)]
            
            # compute histogram
            bow_feature, _ = np.histogram(indices, density=False, bins=bins)
            
            bow_features[i,:] = bow_feature
            
    
        return bow_features

In [116]:
# function to train RF entirely
def train_RF_all(X, y, kfold, max_leaf_nodes, maxFeature, numTrees, criterions):

    # split materials data into tranining and test set
    X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, stratify=y, test_size=0.20)

    trn_A = np.zeros((len(max_leaf_nodes),))
    val_A = np.zeros((len(max_leaf_nodes),))

    # for each maxDepths
    for i, max_leaf_node in enumerate(max_leaf_nodes):      

        # k-fold validation
        skf = StratifiedKFold(n_splits=kfold)
        for lrn_index, val_index in skf.split(X_trn, y_trn):
            X_lrn, X_val = X_trn[lrn_index], X_trn[val_index]
            y_lrn, y_val = y_trn[lrn_index], y_trn[val_index]

            ## perform RF
            trnA, valA, _ = train_RF(X_lrn, y_lrn, X_val, y_val, numTrees, 
                                     maxFeature, max_leaf_nodes=max_leaf_node)
            trn_A[i] += trnA
            val_A[i] += valA
                
    # averaging training and validation accuracy
    trn_A /= kfold
    val_A /= kfold
    
    # print averaged training and validation accuracy
    print('Averaged training Accuracy:')
    print(trn_A)
    print('Averaged validation Accuracy:')
    print(val_A)
    
    # find the optimal model for each cases
    idx = np.where(val_A == val_A.max())
    opt_max_leaf_nodes = max_leaf_nodes[idx[0][0]]

    # calculate class imbalance weight 
    weight = {1 : sum(y_trn==-1)/sum(y_trn==1)}

    ## Training and testing on optimal models
    trn_A, tst_A, opt_model = train_RF(X_trn, y_trn, X_tst, y_tst, numTrees, maxFeature,
                                       max_leaf_nodes=opt_max_leaf_nodes, weights=weight)
    print('Standard RF \nTrain_A: %f \n Test_A: %f\n' % (trn_A, tst_A))

    return opt_model


def train_RF(X_trn, y_trn, X_tst, y_tst, numTree, maxFeature, max_leaf_nodes=3, 
             criterion='gini', weights=None):
    
    model = BalancedRandomForestClassifier(n_estimators=numTree, max_features=maxFeature, max_leaf_nodes=max_leaf_nodes, 
                                           criterion=criterion, n_jobs=15)     
    model.fit(X_trn,y_trn)
    
    ypred = model.predict(X_trn)
    trn_A = accuracy(y_trn, ypred, weights)
    
    ypred = model.predict(X_tst)
    tst_A = accuracy(y_tst, ypred, weights)
    
    return trn_A, tst_A, model


def accuracy(ytrue, ypred, weights=None):
    """Calculate normalized accuracy
    Input:
    ytrue [n,]     : n array containing true y
    ypred [n,]     : n array containing predicted y
    weights [dict] : dict containing the specified class weight. If None, 
                     the weight will be extracted from the trained RF classs.

    Output:
    accuracy [int]: accuracy value
    """

    cls = np.unique(ytrue)     
    if weights == None:
        weight = {cls[0] : 1}
    else:
        weight = weights

    R = list(weight.values())[0]
    tn, fp, fn, tp = confusion_matrix(ytrue, ypred).ravel()

    if [*weight] == cls[0]:  # if key of weight is equal to class 0
        error = (R*fp + fn) / ((fn + tp) + R*(tn + fp))
    else: 
        error = (fp + R*fn) / (R*(fn + tp) + (tn + fp))

    return 1 - error


def predict(model, ids, X_string, X, previous_ypreds=[]):
    
    # find unique id
    ids_int = np.array(list(map(int, ids)))
    unique_ids = np.unique(ids_int)
    
    if len(previous_ypreds) != 0:
        idx_previous_ypreds = np.where(previous_ypreds==1)[0]
        ids_int = np.delete(ids_int, idx_previous_ypreds, axis = 0)
        X = np.delete(X, idx_previous_ypreds, axis = 0)
        X_string = np.delete(X_string, idx_previous_ypreds, axis = 0)
        
    result = []
    
    # predict X test 
    ypreds = model.predict(X) 
    
    # for each unique id, find the 
    for Id in unique_ids:
        idx_subset = np.where(ids_int==Id)[0]
        
        ypred = ypreds[idx_subset]
    
        if sum(ypred) == 0:
            result.append('')
            
        elif sum(ypred) == 1:
            idx = np.where(ypred==1)[0][0]
            result.append(X_string[idx_subset[idx]])
            
        else:
            Idx = np.where(ypred==1)[0]
            counts = np.zeros((len(Idx),))
            for i, idx in enumerate(Idx):
                counts[i] = len(X_string[idx_subset[idx]])
            result.append(X_string[idx_subset[Idx[np.argmax(counts)]]])
            
    return unique_ids, result, ypreds


def write_results_to_csv(n, poi_ids, answers_poi, street_ids, answers_street, file_name):
    empty_strings = [["",""]] * n
    original_df = pd.DataFrame(empty_strings, columns=['POI', 'street'])
    for idx, poi_id in enumerate(poi_ids):
        original_df.iloc[poi_id,0] = answers_poi[idx]
    for idx, street_id in enumerate(street_ids):
        original_df.iloc[street_id,1] = answers_street[idx]

    export_df = pd.DataFrame(empty_strings, columns=['id', 'POI/street'])
    export_df['id'] =original_df.index
    export_df['POI/street'] = original_df[['POI', 'street']].apply(lambda x: '/'.join(x), axis=1)
    export_df.to_csv(file_name, sep=',', encoding='utf-8')

In [9]:
def write_results_to_csv(ids, answers_poi, answers_street, file_name):
    data = []
    for i in range(len(ids)):
        answer_string = answers_poi[i].strip() + "/" + answers_street[i].strip()
        data.append([ids[i], answer_string])
    df = pd.DataFrame(data, columns=['id', 'POI/street'])
    df.to_csv(file_name, sep=',', encoding='utf-8')

# Data Collection

In [10]:
# import training data
df = pd.read_csv('train.csv')

ids = df['id'].values.tolist()
raw_address = df['raw_address'].values.tolist()
answers = df['POI/street'].values.tolist()

df.head()

,id,raw_address,POI/street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung
3,3,"toko dita, kertosono",toko dita/
4,4,jl. orde baru,/jl. orde baru


In [11]:
# import test data
df_test = pd.read_csv('test.csv')

ids_test = df_test['id'].values.tolist()
raw_address_test = df_test['raw_address'].values.tolist()

df_test.head()

,id,raw_address
0,0,s. par 53 sidanegara 4 cilacap tengah
1,1,"angg per, baloi indah kel. lubuk baja"
2,2,"asma laun, mand imog,"
3,3,"ud agung rej, raya nga sri wedari karanganyar"
4,4,"cut mutia, 35 baiturrahman"


# Data Pre-processing

In [ ]:
### tuning parameter ###

# number of K-means cluster
n_cluster = 3000

# sample size ratio
ratio = 0.25

########################

In [12]:
# preprocessing train step
helper = PreprocessingHelper()
X_ngram, y_ngram = helper.preprocesing_step(ids, raw_address, answers)
X_ngram = X_ngram[:,1]

In [13]:
# preprocessing test step
X_ngram_test, _ = helper.preprocesing_step(ids_test, raw_address_test, [])
X_ids_test = X_ngram_test[:,0]
X_ngram_test = X_ngram_test[:,1]

In [14]:
# combine training and test input samples to build unique word dictionary
X_ngram_dictionary = np.concatenate((X_ngram, X_ngram_test))

# perform features reduction via K-mean clustering
bow = bag_of_word_encoder(kmean_n = n_cluster)
bow.build_dictionary(X_ngram_dictionary, load_kmean=False)

X = bow.compute_bow(X_ngram)

# compute Kmeans for test set
X_test = bow.compute_bow(X_ngram_test)

Begin computing K-Means clustering...Complete!


In [15]:
# separate y labels
y_POI = y_ngram[:,0]
y_sn  = y_ngram[:,1]

# reduce training set
_, X_POI_new, _, y_POI_new = train_test_split(X, y_POI, stratify=y_POI, test_size=ratio)

# reduce test set
_, X_sn_new, _, y_sn_new = train_test_split(X, y_sn, stratify=y_sn, test_size=ratio)

In [16]:
# remove variables
del X_ngram_dictionary, df, ids, raw_address, answers, X_ngram, y_ngram, bow

# Model Training

## Perform Random Forest

In [21]:
# hyper-parameter
kfold = 10
max_leaf_nodes = [3000,5000,8000,10000,15000,20000,30000,50000,70000]
maxFeature = int(np.sqrt(X_POI_new.shape[1]))
numTrees = 200
criterions = 'gini'

### POI problem

In [22]:
# train classification
POI_model = train_RF_all(X_POI_new, y_POI_new, kfold, max_leaf_nodes, maxFeature, numTrees, criterions)

Averaged training Accuracy:
[0.7573245  0.76452191 0.77191332 0.77550945 0.78243037 0.78436049
 0.78354664 0.78353235]
Averaged validation Accuracy:
[0.75251857 0.75751394 0.76229055 0.76452176 0.76901865 0.76879858
 0.76649578 0.76654748]
Standard RF 
Train_A: 0.774615 
 Test_A: 0.767309



In [17]:
# save model
pickle.dump(POI_model, open('model_POI.sav', 'wb'))
#POI_model = pickle.load(open('model_POI.sav', 'rb')) # to load model

### Street name problem

In [18]:
# train classification
SN_model = train_RF_all(X_sn_new, y_sn_new, kfold, max_leaf_nodes, maxFeature, numTrees, criterions)

Averaged training Accuracy:
[0.73361242 0.73234516 0.73247273 0.73251883 0.73243222 0.73248746]
Averaged validation Accuracy:
[0.71426981 0.71000094 0.71013749 0.71012689 0.70981269 0.70985379]
Standard RF 
Train_A: 0.710871 
 Test_A: 0.698670



In [20]:
pickle.dump(SN_model, open('model_SN.sav', 'wb'))

# Model Testing

In [114]:
# predict test POI 
print('Begin predict POI test...', end='')
idx_POI, result_POI, ypreds_POI = predict(POI_model, X_ids_test, X_ngram_test, X_test)
print('end!')

# predict test SN
print('Begin predict SN test...', end='')
idx_SN, result_SN, _ = predict(SN_model, X_ids_test, X_ngram_test, X_test, ypreds_POI)
print('end!')

Begin predict POI test...end!
Begin predict SN test...end!


In [117]:
# save result 
write_results_to_csv(df_test.shape[0], idx_POI, result_POI, idx_SN, result_SN, 'test_prediction_result.csv')

In [111]:
type(X_ids_test)

numpy.ndarray